In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
#https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/datasets/uktradecountrybycommodityexports
url = "https://www.ons.gov.uk/file?uri=/economy/nationalaccounts/balanceofpayments/datasets/uktradecountrybycommodityexports/current/countrybycommodityexports.zip"

In [3]:
r = requests.get(url) 

brexit_FEB21 = zf.ZipFile(io.BytesIO(r.content))

brexit_FEB21.extractall(".\\data")

filename = brexit_FEB21.namelist()

In [4]:
df = pd.read_excel(".\\data" + "\\" + filename[0], sheet_name= "Country by Commodity Export")

In [5]:
df.head()

,COMMODITY,COUNTRY,DIRECTION,1997JAN,1997FEB,1997MAR,1997APR,1997MAY,1997JUN,1997JUL,...,2020SEP,2020OCT,2020NOV,2020DEC,2021JAN,2021FEB,2021MAR,2021APR,2021MAY,2021JUN
0,0 Food & live animals,AD Andorra,EX Exports,0.01,0.04,0.07,0.01,0.04,0.04,0.03,...,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00
1,0 Food & live animals,AE United Arab Emirates,EX Exports,4.18,3.93,4.26,4.61,3.77,4.90,3.80,...,19.83,22.09,19.18,22.46,15.02,18.16,21.55,13.20,22.65,16.82
2,0 Food & live animals,AF Afghanistan,EX Exports,2.30,0.00,0.00,0.00,0.01,0.00,0.00,...,0.11,0.12,0.06,0.44,0.13,0.25,0.03,0.02,0.08,0.13
3,0 Food & live animals,AG Antigua & Barbuda,EX Exports,0.17,0.13,0.22,0.27,0.19,0.23,0.32,...,0.19,0.27,0.23,0.30,0.23,0.27,0.29,0.21,0.17,0.17
4,0 Food & live animals,AI Anguilla,EX Exports,0.00,0.00,0.01,0.03,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [6]:
df = df.melt(id_vars = ["COMMODITY", "COUNTRY", "DIRECTION"])

df.drop("DIRECTION", axis = 1, inplace = True)

df.variable= pd.to_datetime(df.variable, format="%Y%b")

df.rename({"variable":"time", "value": "exports"},inplace = True, axis = 1)

df.COUNTRY = df.COUNTRY.str[3:]

df.exports = df.exports*1000000

In [7]:
grp = df[df.COMMODITY == 'T Total'].groupby("COUNTRY")

test = grp.agg({"exports": "sum"})

In [8]:
topcntry = test.sort_values(by = "exports", ascending = False).iloc[0:23].index.to_list()

In [9]:
df = df[df.COUNTRY.isin(topcntry)]

In [10]:
df.COMMODITY.replace({"T Total": "Total"}, inplace = True)
df.COUNTRY.replace({"United States inc Puerto Rico": "United States"}, inplace = True)
df.COUNTRY.replace({"Extra EU 28 (Rest of World)": "Excl. EU 28 (Rest of World)"}, inplace = True)

In [11]:
df

,COMMODITY,COUNTRY,time,exports
1,0 Food & live animals,United Arab Emirates,1997-01-01,4.180000e+06
12,0 Food & live animals,Australia,1997-01-01,4.950000e+06
15,0 Food & live animals,Total EU(28),1997-01-01,3.850200e+08
19,0 Food & live animals,Belgium,1997-01-01,2.255000e+07
36,0 Food & live animals,Canada,1997-01-01,8.180000e+06
...,...,...,...,...
8709697,Total,Sweden,2021-06-01,3.210000e+08
8709698,Total,Singapore,2021-06-01,3.990000e+08
8709724,Total,Turkey,2021-06-01,4.080000e+08
8709732,Total,United States,2021-06-01,3.764000e+09


In [12]:
grp = df.groupby(["COUNTRY"])

In [13]:
def ex_metal_total(df):
        
    if not (df[df["COMMODITY"] == "9 Unspecified goods"].empty):
                
        foo = pd.DataFrame([])

        fooall = df[df["COMMODITY"] == "Total"].set_index(["time"])["exports"] 
        
        foobar = df[df["COMMODITY"] == "9 Unspecified goods"].set_index(["time"])["exports"]

        foo["exports"] = fooall - foobar
    # Grab the total and then sbutract off the unspecified goods
    
        foo["COUNTRY"] = df.COUNTRY.unique()[0]
    
        #print(df.COUNTRY.unique())
    
        foo["COMMODITY"] = "Total (ex. metals)"
        
        foo.reset_index(inplace = True)
        
        foo[df.columns]
        
        df = df.append(foo)
    
    return df

In [14]:
df = grp.apply(ex_metal_total)

In [15]:
df.drop(["COUNTRY"], axis = 1, inplace = True)

df.reset_index(inplace = True)

df.drop(["level_1"], axis = 1, inplace = True)

In [16]:
df.set_index(["COUNTRY", "COMMODITY", "time"], inplace = True)

In [17]:
df.head()

exports
COUNTRY   COMMODITY                   time                 
Australia 0 Food & live animals       1997-01-01  4950000.0
          00 Live animals             1997-01-01  1130000.0
          01 Meat & meat preparations 1997-01-01        0.0
          02 Dairy products & eggs    1997-01-01   140000.0
          03 Fish & shellfish         1997-01-01   360000.0

In [18]:
out_file = ".\\data"+ "\\UK-exports.parquet"

pq.write_table(pa.Table.from_pandas(df.sort_index()), out_file)

In [19]:
file = ".\\data"+ "\\UK-exports.parquet"

foo = pq.read_table(file).to_pandas()

In [20]:
foo.tail()

exports
COUNTRY     COMMODITY          time                    
Whole world Total (ex. metals) 2021-02-01  2.359800e+10
                               2021-03-01  2.901900e+10
                               2021-04-01  2.521900e+10
                               2021-05-01  2.634300e+10
                               2021-06-01  2.718800e+10